深度学习席卷了自然语言处理（natural language processing, NLP）领域，尤其是通过使用不断消耗输入和模型先前输出相结合的模型。这种模型称为递归神经网络（recurrent neural networks, RNN），它已被成功应用于文本分类、文本生成和自动翻译系统。在这之前的NLP工作的特点是复杂的多阶段处理流程，包括编码语言语法的规则。

在本章中，你的目标是将文本转换成神经网络可以处理的东西，就像前面的例子一样，即数值张量。在处理成数值张量之后，再为你的文本处理工作选择正确的网络结构，然后就可以使用PyTorch进行NLP了。你马上就会看到此功能的强大之处：如果你以正确的形式提出了问题，就可以使用相同的PyTorch工具在不同领域中的任务上达到目前最先进的性能。这项工作的第一部分是重塑数据。

网络在两个级别上对文本进行操作：在字符级别上，一次处理一个字符；而在单词级别上，单词是网络中最细粒度的实体。无论是在字符级别还是在单词级别操作，将文本信息编码为张量形式的技术都是相同的。这种技术没什么神奇的，你之前已经用过了，即独热编码。

---
### 开始

加载Jane Austen的《傲慢与偏见》。保存文件并读入文件。

In [5]:
import numpy as np
import pandas as pd
import torch

In [6]:
dset_path = r".\data\chapter3\1342-0.txt"

In [26]:
# np.loadtxt(dset_path,encoding='utf-8')   # 错误: 未知原因

In [27]:
with open(file=dset_path,mode='r',encoding='utf-8') as f:    # 此处gbk格式读取将出错
    text = f.read()

### 对文本中的字符操作，并为每个字符进行独热编码

In [28]:
lines = text.split('\n') #等价于text.splitlines()
line = lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [86]:
# 做一个测试
ord('a'),ord('b') #将转为ascii对应的编码

(97, 98)

In [69]:
# 创建一个零张量，其每一行都要能容纳独热编码字符 
    #备注： 按照 ASCII 有 128个字符   
        # 所以每个字符的独热编码有128位 (抛弃ASCII未编码的字符)
letter_tensor = torch.zeros(len(line),128)  


# 为上面的零张量在正确的位置上设置 1
for i,letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0
    letter_tensor[i][letter_index] = 1
    
letter_tensor.shape

torch.Size([70, 128])

### 通过建立词汇表来在词级别（word-level）对句子（即词序列）进行独热编码

In [78]:
# 测试
    #strip()只能去除两边的指定符号，中间的不可以去除
"123？".strip('？'),"12？3".strip('？')

('123', '12？3')

In [80]:
# 定义clean_words函数，其接受文本，并返回小写，并删除无关符号，比如标点。
def clean_words(input_str):
    word_list = input_str.lower().replace('\n',' ').split()
    word_list = [word.strip('.,;:"!?”“_-')  for word in word_list]
    return word_list

In [85]:
words_in_line = clean_words(line)

line,words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [87]:
# 做一个测试
    # 枚举
{ word:i for (i,word) in enumerate(['i','like','you',])}

{'i': 0, 'like': 1, 'you': 2}

### 经过以上的对一行文本的分析。
现在用同样的方式来对整个文本操作，得到整个文本的单词列表，并且为这些单词建立索引。

In [95]:
# 测试
    # set()转化为集合 
        #sort 与 sorted 区别：sort 是应用在 list 上的方法，sorted 可以对所有可迭代的对象进行排序操作。
set(['a','e','d']),sorted(set(['a','e','d']))

({'a', 'd', 'e'}, ['a', 'd', 'e'])

In [106]:
word_list = sorted(set(clean_words(text)))
word2index_dict = {word:i for i,word in enumerate(word_list)}

# word2index_dict是一个字典，其中单词作为键，而整数作为值。
    # 独热编码时，你将使用此词典来有效地找到单词的索引
len(word2index_dict),word2index_dict['good']  # 显示有7261个单词，单词good的索引是2940

(7261, 2940)

### 继续对一个句子中的单词(词序列)操作，进行独热编码
（再早之前操作是正对字符进行操作，包括对字符的独热编码）
现在对单词进行独热编码，独热编码的位数将不再是128位，而是 len(word2index_dict) 位。

In [130]:
# 现在是对文本中任意句子的分析。将为任意句子中的单词进行独热编码。
    # 建立零张量，该张量的行应该能容纳 len(word2index_dict) 位的独热编码。

word_tensor = torch.zeros(len(line),len(word2index_dict))
for i,word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_tensor[i][word_index] = 1
    print( "{:3} 单词:{:12} 词索引:{:5} 独热:{}".format(i,word,word_index,word_tensor[i]))
    
    
word_tensor.shape

  0 单词:impossible   词索引: 3394 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])
  1 单词:mr           词索引: 4305 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])
  2 单词:bennet       词索引:  813 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])
  3 单词:impossible   词索引: 3394 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])
  4 单词:when         词索引: 7078 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])
  5 单词:i            词索引: 3315 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])
  6 单词:am           词索引:  415 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])
  7 单词:not          词索引: 4436 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])
  8 单词:acquainted   词索引:  239 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])
  9 单词:with         词索引: 7148 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])
 10 单词:him          词索引: 3215 独热:tensor([0., 0., 0.,  ..., 0., 0., 0.])


torch.Size([70, 7261])

---
### 3.1.1 文本嵌入

独热编码是一种将类别数据表示成张量的很有用技术。就像你可能预料到的那样，当需要编码的项目数很大（例如语料库中的单词）时，独热编码就开始崩溃了。一本书中有超过7000个单词！

当然，你可以做一些工作来对单词进行去重、压缩替代拼写、将过去和将来时统一为相同表示，等等。尽管如此，通用的英文编码仍将是巨大的。更糟糕的是，每次遇到一个新单词时，都必须在向量中添加一个新列，这意味着要在模型中添加一组新的权重以解决该新词汇输入问题，从训练角度看这将给你带来很大的痛苦。

如何将编码压缩为更易于管理的大小，并限制大小增长？好吧，可以使用浮点数向量，而不是使用多个0和一个1的向量。举例来说，一个含100个浮点数的向量就可以表示很大量的词汇。**关键是找到一种有效的方法，以一种有助于下游学习的方式将单个单词映射到这个100维空间。这种技术称为嵌入（embedding）**。

原则上，你可以遍历词汇表并为每个单词生成100个随机浮点数。 这种方法可能是有效的，因为你可以将大量词汇塞入100个数字中，但是它会丢弃掉基于语义或上下文的单词之间的任何距离信息。使用这种词嵌入的模型不得不处理其输入向量中的少量结构。**理想的解决方案是以这样的方式生成嵌入：用于同一上下文的单词映射到嵌入空间的邻近区域。**

如果要手工设计解决此问题的方法，你有可能决定通过沿轴映射基本名词和形容词来构建嵌入空间。你可以生成一个二维空间，在该空间中，两个坐标轴分别映射到名词“水果”（0.0-0.33）、“花”（0.33-0.66）和“狗”（0.66-1.0），以及形容词“红色”（0.0-0.2）、“橙色”（0.2-0.4）、“黄色”（0.4-0.6）、“白色”（0.6-0.8）和“棕色”（0.8-1.0）。你现在的目标是将水果、花和狗放置在嵌入中。

开始嵌入单词时，可以将“苹果”映射到“水果”和“红色”象限中的某个数。同样，你可以轻松地映射“橘子”、“柠檬”、“荔枝”和“猕猴桃”（五颜六色的水果）。然后，你可以从花开始，分配“玫瑰”、“罂粟”、“水仙花”、“百合”和...好吧，不存在很多棕色的花。好，“太阳花”可以推出“花”、“黄色”和“棕色”，而“雏菊”可以推出“花”、“白色”和“黄色”。也许你应该更新“猕猴桃”以将其映射到“水果”、“棕色”和“绿色”附近。对于狗和颜色，“redbone（译者注：狗的品种）”、“fox”可能是“橙色”、“金毛”和“贵宾犬”可是“白色”的，以及...大多数种类的狗都是“棕色”的。

尽管对于大型语料库而言，手动进行此映射并不可行，但你应注意，尽管嵌入大小仅为2，但你描述了除基数8个之外的15个不同的单词，如果你花一些创造性的时间，可能还会嵌入更多的单词。

你可能已经猜到了，这种工作是可以自动进行的。**通过处理大量文本语料库，你可以生成与此类似的嵌入。 主要区别在于嵌入向量具有100到1000个元素，并且坐标轴不直接映射到某个词义，但是意思相近的词映射到嵌入空间也是相近的，其轴可能是任意的浮点维（floating-point dimensions）。**
尽管实际使用的算法（比如word2vec）对于我们在此要关注的内容来说有点超出范围，但值得一提的是，**嵌入通常是使用神经网络并试图根据句中邻近词（上下文）预测某个词而生成的。在这种情况下，你可以从独热编码的单词开始，使用（通常是相当浅的）神经网络来生成嵌入。当嵌入可用时，你就可以将其用于下游任务。**

**生成的嵌入的一个有趣的方面是，相似的词不仅会聚在一起，还会与其他词保持一致的空间关系。如果你要使用“苹果”的嵌入向量，并加上和减去其他词的嵌入向量，就可以进行类比，例如苹果 - 红色 - 甜 + 酸，最后可能得到一个类似“柠檬”的向量。**

我们不会在这里使用文本嵌入，但是当必须用数字向量表示集合中的大量元素时，它们是必不可少的工具。